## Import TensorFlow and other libraries

In [1]:
import glob
import cv2
import numpy as np
import os
import shutil
from tqdm import tqdm
from PIL import Image
import random

os.getcwd()

'd:\\UAL\\GANmapper\\datasets'

In [13]:
!python train.py --dataroot .datasets/Exp5/Prepped/Paris15 --name Paris15 --model pix2pix --netG resnet_9blocks

python: can't open file 'train.py': [Errno 2] No such file or directory


## Data Preprocessing

In [5]:
def load_img(target_path, source_path):
    files_target = glob.glob(target_path + '\*\*\*.png', recursive=True)
    files_source = glob.glob(source_path + '\*\*\*.png', recursive=True)
    # files_source = []
    # for i in files_target:
    #     files_source.append(i.replace('Target', 'Black')) 
    print(str(len(files_target)) + ' target files found')
    print(str(len(files_source)) + ' source files found')
    print('Source file example :' + str(files_target[1]))
    print('Source file example :' + str(files_source[1]))
    return files_target, files_source

Singapore: 103.577807083,104.094114302,1.206208429,1.474625026 [EPSG:4326]

# Clean Data
1. Clean source images out of city bounds using landuse Data

In [11]:
# target_path = 'Experiment2\\14\\Target'
# source_path = 'Experiment2\\14\\Source'
# files_target, files_source = load_img(target_path, source_path)

# # find all blank tiles
# rm_list = []
# for file in files_source:
#     img = cv2.imread(file)
#     if np.unique(img, return_counts=True)[1][0] == 786432:
#         rm_list.append(file)

350 target files found
254 source files found
Source file example :Experiment2\14\Target\14\12905\8125.png
Source file example :Experiment2\14\Source\14\12905\8125.png


In [9]:
# # get list of all blank tiles in all folders
# rm_black = []
# rm_color = []
# rm_target = []
# for i in rm_list:
#     rm_black.append(i.replace('Landuse', 'Black')) 
#     rm_color.append(i.replace('Landuse', 'Color')) 
#     rm_target.append(i.replace('Landuse', 'Target')) 
# print('Black file example :' + str(rm_black[1]))
# print('Color file example :' + str(rm_color[1]))
# print('Targe file example :' + str(rm_target[1]))

Black file example :Experiment2\14\Source\14\12905\8134.png
Color file example :Experiment2\14\Source\14\12905\8134.png
Targe file example :Experiment2\14\Source\14\12905\8134.png


In [10]:
# # remove all blank tiles
# all_list = zip(rm_list, rm_black, rm_color, rm_target)
# for f in all_list:
#     os.remove(f[0])
#     # os.remove(f[1])
#     # os.remove(f[2])
#     # os.remove(f[3])

## Stitch Datasets

In [27]:
def combine_target_mask(mask_file_path, target_file_path, threshold):
    img1 = cv2.imread(mask_file_path)
    img2 = cv2.imread(target_file_path)

    # resize
    img1 = cv2.resize(img1, (256,256))    
    img2 = cv2.resize(img2, (256,256))    

    # only take valid target with sufficient buildings     
    if np.unique(img2, return_counts=True)[1][-1] < threshold:   
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        #create empty matrix
        vis = np.zeros((max(h1, h2), w1+w2,3), np.uint8)
        #combine 2 images
        vis[:h1, :w1,:3] = img1
        vis[:h2, w1:w1+w2,:3] = img2
    else:
        vis = None
    return vis

In [ ]:
# threshold for zoom 16 = 170000
# threshold for zoom 15 = 180000

In [39]:
key = 'Kediri'
target_path = 'Exp5\\'+ key +'\\Target'
source_path = target_path.replace('Target', 'Source')
files_target, files_source = load_img(target_path, source_path)

20 target files found
20 source files found
Source file example :Exp5\Kediri\Target\17\106316\68390.png
Source file example :Exp5\Kediri\Source\17\106316\68390.png


In [40]:
count = 0
output_folder = 'Exp5/Prepped/' + key +'/'
try:
    os.makedirs(output_folder)
except:
    pass
for i, x in enumerate(zip(files_source, files_target)):
    # if i >= 50: break
    source, target = x
    output = combine_target_mask(source, target, 170000)
    if output is not None:
        cv2.imwrite(output_folder + str(i) + '.png', output)
        count += 1
print(str(count) + ' out of ' + str(len(files_source)) + ' files processed')

18 out of 20 files processed


In [41]:
# test output
output = combine_target_mask(files_source[888], files_target[888],160000)
img = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
im_pil = Image.fromarray(img)
im_pil.show()

IndexError: list index out of range

## 3. train-test-val split

In [42]:
def train_test_split(file_list, train_size = 0.7):
    random.Random(123).shuffle(file_list)
    train_stop = int(len(file_list)*train_size)
    test_stop = int((len(file_list) - train_stop)/2)
    train_data = file_list[:train_stop]
    test_data = file_list[train_stop: train_stop + test_stop]
    val_data = file_list[train_stop + test_stop:]
    return train_data, test_data, val_data

In [43]:
file_list = glob.glob(output_folder + '/*.png')
train_data, test_data, val_data = train_test_split(file_list, train_size = 0.95)

In [44]:
for i in train_data:
    if not os.path.exists(output_folder+'/train'):
        os.makedirs(output_folder+'/train')
    shutil.move(i, output_folder+'/train')

for i in test_data:
    if not os.path.exists(output_folder+'/test'):
        os.makedirs(output_folder+'/test')
    shutil.move(i, output_folder+'/test')

for i in val_data:
    if not os.path.exists(output_folder+'/val'):
        os.makedirs(output_folder+'/val')
    shutil.move(i, output_folder+'/val')

# for i in file_list:
#     if not os.path.exists(output_folder+'/full'):
#         os.makedirs(output_folder+'/full')
#     shutil.move(i, output_folder+'/full')

In [97]:
#remove road pixels
img1 = "D:\\UAL\\GANmapper Repo\\results\\Beijing\\test_latest\\images\\fake\\16\\53929\\24811.png"

img2 = "D:\\UAL\\GANmapper Repo\\results\\Beijing\\test_latest\\images\\real\\16\\53929\\\\24811.png"

def convert_binary(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh, img = cv2.threshold(img, 128, 255, cv2.THRESH_OTSU)
    return img

img1 = convert_binary(img1)
img2 = convert_binary(img2)
diff = cv2.absdiff(img1, img2)
cv2.imwrite("result.png", img1)
cv2.imwrite("result2.png", img2)
cv2.imwrite("result3.png", diff)

True

In [58]:
ret, bw_img = cv2.threshold(img2,254,255,cv2.THRESH_BINARY)
cv2.imshow("Binary Image",bw_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
diff = cv2.absdiff(img1, img2)
mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

th = 1
imask =  mask>th

canvas = np.zeros_like(img2, np.uint8)
canvas[imask] = img2[imask]
cv2.imwrite("result.png", diff)

In [4]:
# stitch for prediction
key = 'LA'
target_path = 'Exp4\\'+ key +'\\Target'
source_path = target_path.replace('Target', 'Source')
files_target, files_source = load_img(target_path, source_path)

36 target files found
36 source files found
Source file example :Exp4\LA\Target\16\11237\26166.png
Source file example :Exp4\LA\Source\16\11237\26166.png


In [8]:
count = 0
output_folder = 'Exp4/' + key +'/stitched'
try:
    os.makedirs(output_folder)
except:
    pass
for i, x in enumerate(zip(files_source, files_target)):
    # if i >= 50: break
    source, target = x
    output = combine_target_mask(source, target, 170000)
    if output is not None:
        cv2.imwrite(output_folder + str(i) + '.png', output)
        count += 1
print(str(count) + ' out of ' + str(len(files_source)) + ' files processed')

0 out of 36 files processed
